In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import pickle
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout, LSTM
from keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import KFold
import keras
import tensorflow as tf
import time

In [6]:
# data_folder = '/content/gdrive/MyDrive/University/master/term1/Machine-Learning/ML-Project/Data/chroma/'
data_folder = "/content/gdrive/MyDrive/Tehran University/ML/ML-Project/Data/chroma/"

In [7]:
SHAPE_SIZE = (862, 36)

In [8]:
file_name  = data_folder + "mfccaudio_chroma_features_36_all_20Sec_cleaned.pkl"
file_name_labels  = data_folder + "mfccaudio_chroma_features_36_all_20Sec_labels_cleaned.pkl"
with open(file_name, 'rb') as file:
    uncleaned_data = pickle.load(file)

with open(file_name_labels, 'rb') as file:
    uncleaned_labels = pickle.load(file)
np.shape(uncleaned_data), np.shape(uncleaned_labels)

((9538, 862, 36), (9538,))

In [9]:
np.shape(uncleaned_data[0])

(862, 36)

In [10]:
data = []
labels = []
for k, j in zip(uncleaned_data,uncleaned_labels):
    if np.shape(k) != SHAPE_SIZE:
        print(np.shape(k))
    else:
        data.append(k)
        labels.append(j)
labels = np.array(labels)
data = np.array(data)
np.shape(data), np.shape(labels)

((9538, 862, 36), (9538,))

In [11]:
np.unique(uncleaned_labels)

array([0, 1, 2, 3, 4, 5, 6])

In [12]:
uncleaned_labels = []
uncleaned_data = []

In [13]:
import numpy
a = numpy.array(uncleaned_labels)
unique, counts = numpy.unique(a, return_counts=True)
dict(zip(unique, counts))

{}

In [14]:
labels = to_categorical(labels)
# x_train, x_test, y_train, y_test =  train_test_split(data, labels, test_size=0.2, shuffle=True, random_state=42, stratify=labels)

In [15]:
# np.shape(x_train), np.shape(y_train)

In [16]:
class Trainer:
    def __init__(self, number_of_class=7 ,input_size=None, epochs=1000, optimizer='adam'):
        self.input_size = input_size
        self.model = Sequential()
        self.epochs = epochs
        self.optimizer = optimizer
        self.number_of_class = number_of_class
        self.callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100)
    # define cnn model
    def define_model(self):
        print("Build LSTM RNN model ...")
        self.model = Sequential()

        self.model.add(LSTM(units=128, dropout=0.5, return_sequences=True, input_shape=self.input_size))
        self.model.add(LSTM(units=32,  dropout=0.5, return_sequences=False))
        self.model.add(Dense(units=self.number_of_class, activation="softmax"))
        self.model.compile(optimizer=self.optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
        return self.model.summary()

    def fit_model(self, x_train, y_train):
        # fit model
        self.history = self.model.fit(x_train, y_train, epochs=self.epochs, batch_size=128, verbose=1,shuffle=True, 
                                      callbacks=[self.callback])

    def plot_history(self):
        fig = plt.figure(figsize=(12, 4))
        metrics = ['loss', 'accuracy']
        for n, metric in enumerate(metrics):
            plt.subplot(1, 2, n + 1)
            plt.plot(self.history.epoch, self.history.history[metric], label='Train')
            plt.plot(self.history.epoch, self.history.history[f"val_{metric}"], linestyle="--", label='Validation')
            plt.xlabel('Epoch')
            plt.ylabel(metric)
            plt.title(metric)
        plt.legend()
        plt.show()

    def evaluate(self, x_test, y_test):
        y_pred = np.argmax(self.model.predict(x_test), axis=-1)
        print(classification_report(np.argmax(y_test, axis=-1), y_pred, digits=3))



In [17]:

# trainer.define_model()

In [18]:
fold_no = 1
num_folds = 4
kfold = KFold(n_splits=num_folds, shuffle=True)
acc_per_fold = []
loss_per_fold = []
for train, test in kfold.split(data, labels):
    trainer = Trainer(input_size = SHAPE_SIZE, epochs=500)
    trainer.define_model()  
    print('done')
    # trainer.fit_model(data[train], labels[train])
    trainer.model.fit(data[train], labels[train], epochs=trainer.epochs, batch_size=128, verbose=1,shuffle=True, 
                                      callbacks=[trainer.callback])
    scores = trainer.model.evaluate(data[test], labels[test], verbose=0)
    print(f'Score for fold {fold_no}: {trainer.model.metrics_names[0]} of {scores[0]}; {trainer.model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    fold_no += 1 
    del trainer.model
    del trainer
    keras.backend.clear_session()
    tf.keras.backend.clear_session()
    time.sleep(5)
# print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
acc_per_fold

Build LSTM RNN model ...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 862, 128)          84480     
                                                                 
 lstm_1 (LSTM)               (None, 32)                20608     
                                                                 
 dense (Dense)               (None, 7)                 231       
                                                                 
Total params: 105,319
Trainable params: 105,319
Non-trainable params: 0
_________________________________________________________________
done
Epoch 1/500
56/56 [==============================] - 15s 86ms/step - loss: 1.9217 - accuracy: 0.1995
Epoch 2/500
56/56 [==============================] - 5s 86ms/step - loss: 1.8772 - accuracy: 0.2339
Epoch 3/500
56/56 [==============================] - 5s 81ms/step - loss: 1.8570 

[73.29140305519104, 73.58490824699402, 72.5671112537384, 72.65100479125977]

In [19]:
# trainer.plot_history()

In [20]:
# trainer.evaluate(x_test, y_test)

In [21]:
# trainer.evaluate(x_train, y_train)